In [ ]:
%matplotlib
import requests
import pandas as pd
import collections
import csv
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
from itertools import groupby
import seaborn as sns
sns.set()
#plt.style.use('seaborn-whitegrid')


In [ ]:
#ucitavanja podataka koje eksportira lens
url="D:\\Google Drive\\Doktorski\\Datasets\\Car Headlights\\Car Headlights.xlsx"
#url='D:\\Google Drive\\Doktorski\\Datasets\\Datasets 2021\\Bicycle\\Bicycle.xlsx'
#Url za na poslu
"url_p=U:\\Radni\\Datasets\\Blockchain.xlsx"
    
data=pd.read_excel(url)

In [ ]:
#ciscenje podataka i brisanje stupaca koji su bespotrebni
data.drop(['Kind', 'Lens ID','Priority Numbers','URL','Type','Full Text', 'Cited Count', 'Sequence Count'], axis=1)
#Tu bi jos bilo zanimljivo odvojiti klasifikacijske kodove


In [ ]:
#PLOTANJE S KRIVULJE - TLC ANALIZA (mozda dvije analize, po godinama i datumu)
years=data['Publication Year']
years_plot=years.value_counts(sort=False).to_frame()
years_plot['Sum']=0
years_plot.iloc[0,1]=years_plot.iloc[0,0]
for i in range(1, len(years_plot.index)):                                          #petlja koja sumira broj patenata po godini
    years_plot.iloc[i,1]=years_plot.iloc[i-1,1]+years_plot.iloc[i,0]

years_plot.drop('Publication Year', axis=1, inplace=True)    

x=np.arange(0,len(years_plot),1)
fig, ax =plt.subplots(1,1)
ax.plot(x, years_plot['Sum'])
ax.set_xticks(x)
ax.set_xticklabels(years_plot.index.values)
plt.xticks(rotation=90)
plt.show()

In [ ]:
#Kvartali (za nove tehnologije) Treba u tablicu dodat prazne periode 
qua=data['Application Date']
qua_plot=qua.to_frame()
qua_plot['Quarter']=pd.PeriodIndex(qua_plot['Application Date'], freq='Q') #kreiranje kvartala iz datuma
qua_plot=qua_plot['Quarter'].value_counts(sort=False).to_frame()            #count po kvartalu
qua_plot=qua_plot.sort_index()                                             #Sortira po kvartalima

qua_plot['Sum']=0
qua_plot.iloc[0,1]=qua_plot.iloc[0,0]

for i in range(1,len(qua_plot.index)):                                     #sumira broj patenata
    qua_plot.iloc[i,1]=qua_plot.iloc[i-1,1]+qua_plot.iloc[i,0]

qua_plot.drop('Quarter', axis=1, inplace=True)

x=np.arange(0, len(qua_plot),1)
fig, ax=plt.subplots(1,1)
ax.plot(x, qua_plot['Sum'])
ax.set_xticks(x)
ax.set_xticklabels(qua_plot.index.values, rotation='vertical')
plt.show()

In [ ]:
#TESTNI PROZOR
test=data['Application Date']
test=test.to_frame()
test['Quarter']=pd.PeriodIndex(test['Application Date'], freq='Q')
temp=pd.PeriodIndex(start=test['Application Date'][0], end=test['Application Date'][len(test.index)-1], freq='Q')
df=pd.DataFrame(index=temp)
#df['A']=0
df.index.names = ['Quarter']
test=test.set_index('Quarter')
df.index=df.index.to_series().astype(str) #mijenja indekse is period index u string da se mogu spojit
test.index=test.index.to_series().astype(str)

joined=df.join(test, how='outer')
joined['Quarter']=joined.index
joined['Temp']=0

for i in range(0,len(joined.index)):
    if type(joined.iloc[i,0])== pd._libs.tslib.Timestamp:
            joined.iloc[i,2]=1
joined



In [ ]:
#Analiza applicants
s=list()
appli=data['Applicants'].appli.str.split(';;', expand=True)  #Kreira data set sa applicants, 
                                                            # Proširuje (applicants su unutar columna odvojeni s ';;')

for a in range(0, len(appli.index)):  #Kreira listu sa svim applicantima
    for b in range(0, len(appli.columns)):
        s.append(appli.iloc[a,b])
s=pd.Series(list(filter(None, s))) #cisti None is liste i pretvara ju u series

temp=s.value_counts(sort=True).to_frame(name='Count') # Count applicants 
appli_plot=temp.drop(temp[temp.Count < 5].index)      #Ovo iz liste applicanta izbaci sve koji imaju manje od 5 patentata
                                                      # 5 je odabrano proizvoljno, cilj je dobiti uvid u najvece igrace     
    
fix, ax=plt.subplots()                              #plota bar plot Applicant-Count
plt.bar(appli_plot.index, appli_plot['Count'])
plt.xticks(rotation='vertical')   
plt.subplots_adjust(bottom=0.50)                    #osigurava da se ne odreze oznaka applicanta
plt.show()


In [ ]:
#Analiza Inventors (analognlo analizi applicants)

s1=list()
inv=data['Inventors'].str.split(';;', expand=True) 

for a in range(0, len(inv.index)):  #Kreira listu sa svim applicantima
    for b in range(0, len(inv.columns)):
        s1.append(inv.iloc[a,b])
        
s1=pd.Series(list(filter(None, s1)))

temp1=s1.value_counts(sort=True).to_frame(name='Count') 
inv_plot=temp1.drop(temp1[temp1.Count < 5].index)   


fix, ax=plt.subplots()                              
plt.bar(inv_plot.index, inv_plot['Count'])
plt.xticks(rotation='vertical')   
plt.subplots_adjust(bottom=0.50)                    #neki cudni tickovi na y osi, popeglaj to malo
plt.show()


In [ ]:
# klasifikacijski kodovi CPC
cpc=data['CPC Classifications'].str.split(';;', expand=True)


In [ ]:
# klasifikacijski kodovi IPC
ipc=data['IPCR Classifications'].str.split(';;', expand=True)
ipc=ipc.apply(lambda x: x.str.slice(0,4))  #ovo zadrzi samo prvih n (4) znamenaka klasifikacijskog koda

#broj kodova po patentu
ipc['Sum']=ipc.apply(lambda x: x.count(), axis=1)  #sumira broj kodova po patentu
ipc['Sum']=ipc['Sum'].astype(int)


print('Largest number of classification codes per one patent:' , ipc['Sum'].max())
print('Most frequent number of classification codes per one patent', ipc['Sum'].mode())
print('Average number of classification codes per one patent', ipc['Sum'].mean())

fig, ax1=plt.subplots()
ax1.hist(ipc['Sum'], bins=np.arange(ipc['Sum'].min()-1,ipc['Sum'].max()+1), rwidth=0.9)
ax1.set_title('Frequency of number of classfication codes')
ax1.set_ylabel('Frequency')
ax1.set_xlabel('Number of classification codes in patent')
ax1.set_xlim(ipc['Sum'].min(), ipc['Sum'].max())
plt.xticks(range(ipc['Sum'].max()+1))   #mozda bi trebalo pomaknut plot na sredinu
fig.tight_layout()




#Patent breadth: total number of IPC codes/number of examined patents
ipc_set=set()
for a in range(0, len(ipc.index)):
    for b in range(0, len(ipc.columns)):
        ipc_set.add(ipc.iloc[a,b])

ipc_set=pd.Series(list(filter(None, ipc_set)))


